In [34]:
import pandas as pd
import numpy as np

from ISLP import load_data
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [35]:
NYSE = load_data('NYSE')
cols = ['DJ_return', 'log_volume', 'log_volatility']
X = pd.DataFrame(
    StandardScaler(with_mean=True, with_std=True).fit_transform(NYSE[cols]),
    columns=NYSE[cols].columns, index=NYSE.index)

In [36]:
for lag in range(1, 6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

In [37]:
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)
X.columns

Index(['DJ_return_1', 'log_volume_1', 'log_volatility_1', 'DJ_return_2',
       'log_volume_2', 'log_volatility_2', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_4', 'log_volume_4', 'log_volatility_4',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5'],
      dtype='object')

In [38]:
M = LinearRegression()
M.fit(X[train], Y[train])
M.score(X[~train], Y[~train])

0.4128912938562521

In [43]:
X_day = pd.merge(X, pd.get_dummies(NYSE['day_of_week']), on='date')
print(X_day.shape)

(6046, 20)


In [40]:
M.fit(X_day[train], Y[train])
M.score(X_day[~train], Y[~train])

0.4595563133053273

In [41]:
month_df = pd.get_dummies(NYSE.index.to_series().apply(lambda x: x[5:7]))
X_month = pd.merge(X_day, month_df, on='date')

In [42]:
M.fit(X_month[train], Y[train])
M.score(X_month[~train], Y[~train])

0.46268360820427235